In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras as K
import tensorflow as tf
import gzip
import random
%matplotlib inline

In [4]:
train_df =  pd.read_csv('train_1.csv')

In [5]:
print(len(train_df))
train_df.head()

1018061


,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.422380e+19,1,14102100,1005,0,f9c69707,e16ceb4b,28905ebd,ecad2386,7801e8d9,...,1,0,19665,320,50,2253,2,303,100126,52
1,1.422380e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15699,320,50,1722,0,35,-1,79
2,1.422390e+19,0,14102100,1005,0,85f751fd,c4e18dd6,50e219e0,c0c5eb49,f2f777fb,...,1,0,21611,320,50,2480,3,297,100111,61
3,1.422400e+19,0,14102100,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,1,0,20359,216,36,2333,0,39,-1,157
4,1.422470e+19,0,14102100,1005,0,f282ab5a,61eb5bc4,f028772b,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [6]:
test_df =  pd.read_csv('test.csv')

In [7]:
print(len(test_df))
test_df.head()

500578


,id,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.190000e+19,14103105,1005,1,178a7b89,b4598159,f028772b,ecad2386,7801e8d9,07d7df22,...,1,0,17893,320,50,2039,2,39,100081,32
1,1.190000e+19,14103105,1005,0,85f751fd,c4e18dd6,50e219e0,c8e3e3c1,b5f3b24a,8ded1f7a,...,1,2,21666,320,50,2494,3,39,-1,112
2,1.190000e+19,14103105,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,1,0,23141,320,50,2664,0,35,-1,51
3,1.190000e+19,14103105,1005,0,93eaba74,7687a86e,3e814130,ecad2386,7801e8d9,07d7df22,...,1,0,19016,300,250,2162,2,39,-1,33
4,1.190000e+19,14103105,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,1,0,23141,320,50,2664,0,35,-1,51


In [8]:
submission_df = pd.read_csv('sampleSubmission.csv')

In [9]:
def to_date_column(df):
    df["dt_hour"] = pd.to_datetime(df["hour"], format="%y%m%d%H")
    df["year"] = df["dt_hour"].dt.year
    df["month"] = df["dt_hour"].dt.month
    df["day"] = df["dt_hour"].dt.day
    df["int_hour"] = df["dt_hour"].dt.hour
    df["is_weekday"] = df["dt_hour"].dt.dayofweek
    df["is_weekend"] = df.apply(lambda x: x["is_weekday"] in [5, 6], axis=1)

In [10]:
to_date_column(train_df)
train_df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C19,C20,C21,dt_hour,year,month,day,int_hour,is_weekday,is_weekend
0,1.422380e+19,1,14102100,1005,0,f9c69707,e16ceb4b,28905ebd,ecad2386,7801e8d9,...,303,100126,52,2014-10-21,2014,10,21,0,1,False
1,1.422380e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,35,-1,79,2014-10-21,2014,10,21,0,1,False
2,1.422390e+19,0,14102100,1005,0,85f751fd,c4e18dd6,50e219e0,c0c5eb49,f2f777fb,...,297,100111,61,2014-10-21,2014,10,21,0,1,False
3,1.422400e+19,0,14102100,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,39,-1,157,2014-10-21,2014,10,21,0,1,False
4,1.422470e+19,0,14102100,1005,0,f282ab5a,61eb5bc4,f028772b,ecad2386,7801e8d9,...,35,-1,157,2014-10-21,2014,10,21,0,1,False


In [11]:
train_df.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'dt_hour', 'year',
       'month', 'day', 'int_hour', 'is_weekday', 'is_weekend'],
      dtype='object')

In [12]:
train_df.nunique()

id                  492964
click                    2
hour                     6
C1                       7
banner_pos               7
site_id               2093
site_domain           2044
site_category           20
app_id                2339
app_domain             159
app_category            23
device_id            85978
device_ip           320074
device_model          4602
device_type              4
device_conn_type         4
C14                    606
C15                      8
C16                      9
C17                    162
C18                      4
C19                     41
C20                    162
C21                     35
dt_hour                  6
year                     1
month                    1
day                      1
int_hour                 6
is_weekday               1
is_weekend               1
dtype: int64

In [13]:
label_col = "click"
x_columns = set(list(train_df.columns)) - set(["id", "site_id", "app_id", "hour", "dt_hour", "device_id", "device_ip"] + [label_col])

In [14]:
x_train = train_df[x_columns]
y_train = train_df[label_col]


In [15]:
x_train.nunique()

C1                     7
device_type            4
C18                    4
month                  1
C21                   35
device_conn_type       4
device_model        4602
day                    1
site_category         20
C20                  162
is_weekend             1
int_hour               6
C15                    8
C19                   41
app_domain           159
year                   1
site_domain         2044
app_category          23
banner_pos             7
is_weekday             1
C16                    9
C17                  162
C14                  606
dtype: int64

In [16]:
to_date_column(test_df)
x_test = test_df[x_columns]


In [17]:
x_test.head()

,C1,device_type,C18,month,C21,device_conn_type,device_model,day,site_category,C20,...,C19,app_domain,year,site_domain,app_category,banner_pos,is_weekday,C16,C17,C14
0,1005,1,2,10,32,0,76dc4769,31,f028772b,100081,...,39,7801e8d9,2014,b4598159,07d7df22,1,4,50,2039,17893
1,1005,1,3,10,112,2,2c5339cd,31,50e219e0,-1,...,39,b5f3b24a,2014,c4e18dd6,8ded1f7a,0,4,50,2494,21666
2,1005,1,0,10,51,0,c6263d8a,31,28905ebd,-1,...,35,7801e8d9,2014,f3845767,07d7df22,0,4,50,2664,23141
3,1005,1,2,10,33,0,1f0bc64f,31,3e814130,-1,...,39,7801e8d9,2014,7687a86e,07d7df22,0,4,250,2162,19016
4,1005,1,0,10,51,0,c6263d8a,31,28905ebd,-1,...,35,7801e8d9,2014,f3845767,07d7df22,0,4,50,2664,23141


In [18]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder


x_train_len = len(x_train)
n_df = pd.concat([x_train, x_test])

In [19]:
d = defaultdict(LabelEncoder)
n_df = n_df.apply(lambda x: d[x.name].fit_transform(x))
n_df.head()

,C1,device_type,C18,month,C21,device_conn_type,device_model,day,site_category,C20,...,C19,app_domain,year,site_domain,app_category,banner_pos,is_weekday,C16,C17,C14
0,2,1,2,0,13,0,2468,0,1,92,...,18,85,0,2235,0,0,0,2,101,377
1,2,1,0,0,19,0,2178,0,1,0,...,1,85,0,2393,0,0,0,2,45,206
2,2,1,3,0,14,0,966,0,5,82,...,16,171,0,1948,3,0,0,2,161,623
3,2,1,0,0,35,0,2575,0,18,0,...,2,85,0,1238,0,1,0,1,127,476
4,2,1,0,0,35,0,581,0,18,0,...,1,85,0,956,0,0,0,2,90,343


In [20]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
scaler.fit(n_df)
n_df = scaler.transform(n_df)
len(n_df)

1518639

In [21]:
n_x_train = n_df[:x_train_len]
n_x_test = n_df[x_train_len:]

In [22]:
from sklearn.linear_model import LogisticRegression


assert len(n_x_train) == len(y_train)
clf = LogisticRegression(verbose=True)
clf.fit(n_x_train, y_train)


y_train_star = clf.predict(n_x_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.7s finished


In [23]:
from sklearn.metrics import log_loss


log_loss(y_train, y_train_star)

5.523227653985857

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

accuracy_score(y_train, y_train_star)

0.8400862030860626

In [25]:
confusion_matrix(y_train, y_train_star)

array([[854933,    993],
       [161809,    326]], dtype=int64)

In [26]:
from sklearn.ensemble import RandomForestClassifier

assert len(n_x_train) == len(y_train)
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
model.fit(n_x_train, y_train)

y_train_star_1 = model.predict(n_x_train)

In [27]:
from sklearn.metrics import log_loss


log_loss(y_train, y_train_star_1)

3.9842858454553505

In [28]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

accuracy_score(y_train, y_train_star_1)

0.8846434545670643

In [29]:
confusion_matrix(y_train, y_train_star_1)

array([[840753,  15173],
       [102267,  59868]], dtype=int64)

In [30]:
from sklearn.linear_model import SGDClassifier

assert len(n_x_train) == len(y_train)
sdc = SGDClassifier(loss="hinge", penalty="l2", max_iter=100)
sdc.fit(n_x_train, y_train)

y_train_star_sdc = sdc.predict(n_x_train)

In [31]:
from sklearn.metrics import log_loss


log_loss(y_train, y_train_star_sdc)

5.500598206579798

In [32]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

accuracy_score(y_train, y_train_star_sdc)

0.8407413701143645

In [33]:
confusion_matrix(y_train, y_train_star_sdc)

array([[855926,      0],
       [162135,      0]], dtype=int64)

In [34]:
y_test = clf.predict(n_x_test)

In [35]:
y_test = clf.predict_proba(n_x_test)
print(clf.classes_)
print(y_test.shape)
print(y_test[3:5, ])

y_test = y_test[:, 1]
print(y_test.shape)
print(y_test[3:5])

[0 1]
(500578, 2)
[[0.05907237 0.94092763]
 [0.43039941 0.56960059]]
(500578,)
[0.94092763 0.56960059]
